In [1]:
# Libraries
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 40)
pd.set_option('display.width', 2000)
import torch
import pickle

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# Import
path = r'switch_data/final_trimmed_snomed_embedding_128d-copy.csv'
embedding = pd.read_csv(path)

# Import
path = r'switch_data/final_problem_dummies.csv'
problem_dummies = pd.read_csv(path)

In [3]:
# Import
path = r'switch_data/icare_switch_data.csv'
icare_df = pd.read_csv(path)
# Import
path = r'switch_data/icare_switch_data_preprocessed_2023.csv'
icare_df_2023 = pd.read_csv(path)
# Concat
joint_switch = pd.concat([icare_df, icare_df_2023])

In [2]:
import json, snowflake.connector

# establish the connection to snowflake
ctx = snowflake.connector.connect( 
    **json.load(open('/opt/ich/python-snowflake-defaults.json')))
    
# verify and test if connection is working
try: 
    cs = ctx.cursor() 
    cs.execute('SELECT current_version(), current_role(), current_warehouse()')
    print(cs.fetchone())
finally: 
    cs.close()

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/snowflake/connector/options.py:96: UserWarning: You have an incompatible version of 'pyarrow' installed (12.0.1), please install a version that adheres to: 'pyarrow<8.1.0,>=8.0.0; extra == "pandas"'
  warn_incompatible_dep(


Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
We were unable to open a browser window for you, please open the following url manually then paste the URL you are redirected to into the terminal.
URL: https://login.microsoftonline.com/be00e2c6-806c-45f8-84c3-3fc99f64b8d3/saml2?SAMLRequest=pZNLb%2BIwFEb%2FSuRZJ3EeQLAIFVMEQ1VaxKOL2QXnhlokNvg6BP59TShSZ9FuRsrCss%2B1j%2B%2FnDB7OVemcQKNQMiWBR4kDkqtcyF1KNuuJmxAHTSbzrFQSUnIBJA%2FDAWZVeWCj2rzLJRxrQOPYjSSydiEltZZMZSiQyawCZIaz1Wj%2BzEKPsoNWRnFVki8lP1dkiKCNNbyX5Cis3rsxB%2Bb7TdN4TeQpvfNDSqlP%2B76lrsivO3%2B2d%2FqGD3waX3lLWHzx6fZbyFsLftLa3iBkf9brhbt4Xa2JM7qrPiqJdQV6BfokOGyWzzcBtAaIUS9Jul69d1HZHnoHLU6ZgVLIvYdSNUWZ7YGr6lAbe4BnR34BuV%2BqnbA9mI1TctiL%2FPK0OQF%2FfRvT4yo%2BC3jqTabdTtQLl8fpselcjIpf5s20mIPixHm7hxxeQ54h1jCT12iNnaJh7NLADcN1ELHrF3jdfu8vccY2WiEz01be%2FVsPrxJcK1SF

('8.2.0', 'ICHT_RG_COVOAM_22016_PROD', 'ICHT_WH_COVOAM_22016_PROD')


True

In [6]:
# Import 
query = '''
SELECT * from ICHT_PROD.ICHT_COVID.EPISODES
'''
cur = ctx.cursor().execute(query)
episodes = pd.DataFrame.from_records(iter(cur), columns=[x[0] for x in cur.description])

In [5]:
# Import 
query = '''
SELECT * from ICHT_PROD.ICHT_COVID.EPISODES_2023
'''
cur = ctx.cursor().execute(query)
episodes_2023 = pd.DataFrame.from_records(iter(cur), columns=[x[0] for x in cur.description])

In [15]:
# Create lists 
subject_list = problem_dummies['SUBJECT'].unique().tolist()
spell_list = joint_switch['SPELL_IDENTIFIER'].unique().tolist()
# Filter 
episodes = episodes[episodes['SUBJECT'].isin(subject_list)]
episodes = episodes[episodes['SPELL_IDENTIFIER'].isin(spell_list)]
episodes_2023 = episodes_2023[episodes_2023['SUBJECT'].isin(subject_list)]
episodes_2023 = episodes_2023[episodes_2023['SPELL_IDENTIFIER'].isin(spell_list)]
# Drop duplicates
episodes.drop_duplicates(subset=['SUBJECT', 'SPELL_IDENTIFIER'], inplace=True)
episodes_2023.drop_duplicates(subset=['SUBJECT', 'SPELL_IDENTIFIER'], inplace=True)

In [8]:
len(subject_list)
len(spell_list)
episodes.SUBJECT.nunique()
episodes.SPELL_IDENTIFIER.nunique()
episodes_2023.SUBJECT.nunique()
episodes_2023.SPELL_IDENTIFIER.nunique()

65160

12251

4853

5610

5176

6094

In [10]:
len(set.intersection(set(episodes['SUBJECT']), set(episodes_2023['SUBJECT'])))
len(set.intersection(set(episodes['SPELL_IDENTIFIER']), set(episodes_2023['SPELL_IDENTIFIER'])))

4792

5547

In [22]:
# Now have unique spell to subject mapping

In [30]:
# Import 
query = '''
SELECT * from ICHT_PROD.ICHT_COVID.EPISODES_DIAGNOSIS
'''
cur = ctx.cursor().execute(query)
diagnosis = pd.DataFrame.from_records(iter(cur), columns=[x[0] for x in cur.description])

In [32]:
# Filter
diagnosis = diagnosis[diagnosis['SUBJECT'].isin(subject_list)]
diagnosis = diagnosis[diagnosis['SPELL_IDENTIFIER'].isin(spell_list)]

In [37]:
diagnosis.SUBJECT.nunique()
diagnosis.SPELL_IDENTIFIER.nunique()

4847

5603

In [41]:
# Filter by uti
uti_diagnoses = diagnosis[(diagnosis['DIAGNOSIS_DESCRIPTION_ICD'].str.contains("urinary tract infection", case=False)) | \
                                  (diagnosis['DIAGNOSIS_DESCRIPTION_ICD'].str.contains("pyelonephritis", case=False)) | \
                                  (diagnosis['DIAGNOSIS_DESCRIPTION_SNOMED'].str.contains("urinary tract infection", case=False)) | \
                                  (diagnosis['DIAGNOSIS_DESCRIPTION_SNOMED'].str.contains("pyelonephritis", case=False))]
# Filter by pneumonia
pneumonia_diagnoses = diagnosis[(diagnosis['DIAGNOSIS_DESCRIPTION_ICD'].str.contains("pneumonia", case=False)) | \
                                  (diagnosis['DIAGNOSIS_DESCRIPTION_SNOMED'].str.contains("pneumonia", case=False))]

# Filter by sepsis
sepsis_diagnoses = diagnosis[(diagnosis['DIAGNOSIS_DESCRIPTION_ICD'].str.contains("sepsis", case=False)) | \
                                  (diagnosis['DIAGNOSIS_DESCRIPTION_SNOMED'].str.contains("sepsis", case=False))]

# Filter by cellulitis
cellulitis_diagnoses = diagnosis[(diagnosis['DIAGNOSIS_DESCRIPTION_ICD'].str.contains("cellulitis", case=False)) | \
                                  (diagnosis['DIAGNOSIS_DESCRIPTION_SNOMED'].str.contains("cellulitis", case=False))]

# Filter by bacteraemia
bacteraemia_diagnoses = diagnosis[(diagnosis['DIAGNOSIS_DESCRIPTION_ICD'].str.contains("blood stream infection", case=False)) | \
                                  (diagnosis['DIAGNOSIS_DESCRIPTION_ICD'].str.contains("bacteraemia", case=False)) | \
                                  (diagnosis['DIAGNOSIS_DESCRIPTION_SNOMED'].str.contains("blood stream infection", case=False)) | \
                                  (diagnosis['DIAGNOSIS_DESCRIPTION_SNOMED'].str.contains("bacteraemia", case=False))]

In [42]:
uti_diagnoses.SUBJECT.nunique()
uti_diagnoses.SPELL_IDENTIFIER.nunique()

pneumonia_diagnoses.SUBJECT.nunique()
pneumonia_diagnoses.SPELL_IDENTIFIER.nunique()

sepsis_diagnoses.SUBJECT.nunique()
sepsis_diagnoses.SPELL_IDENTIFIER.nunique()

cellulitis_diagnoses.SUBJECT.nunique()
cellulitis_diagnoses.SPELL_IDENTIFIER.nunique()

bacteraemia_diagnoses.SUBJECT.nunique()
bacteraemia_diagnoses.SPELL_IDENTIFIER.nunique()

676

735

1354

1504

1566

1781

230

261

0

0

In [82]:
735 + 1504 + 1504 + 261
len(set.union(set(uti_diagnoses.SPELL_IDENTIFIER.unique()), set(pneumonia_diagnoses.SPELL_IDENTIFIER.unique()), set(sepsis_diagnoses.SPELL_IDENTIFIER.unique()), set(cellulitis_diagnoses.SPELL_IDENTIFIER.unique())))
len(set.intersection(set(uti_diagnoses.SPELL_IDENTIFIER.unique()), set(pneumonia_diagnoses.SPELL_IDENTIFIER.unique()), set(sepsis_diagnoses.SPELL_IDENTIFIER.unique()), set(cellulitis_diagnoses.SPELL_IDENTIFIER.unique())))

4004

2973

3

In [ ]:
# Create binary columns
uti_diagnoses['uti'] = 1
pneumonia_diagnoses['pneumonia'] = 1
sepsis_diagnoses['sepsis'] = 1
cellulitis_diagnoses['cellulitis'] = 1

# Merge DataFrames
merged_df = uti_diagnoses[['SPELL_IDENTIFIER', 'uti']]
merged_df = merged_df.merge(pneumonia_diagnoses[['SPELL_IDENTIFIER', 'pneumonia']], how='outer')
merged_df = merged_df.merge(sepsis_diagnoses[['SPELL_IDENTIFIER', 'sepsis']], how='outer')
merged_df = merged_df.merge(cellulitis_diagnoses[['SPELL_IDENTIFIER', 'cellulitis']], how='outer')

# Fill NaN values with 0
merged_df = merged_df.fillna(0)
disease_df = merged_df.drop_duplicates()
disease_df.reset_index(inplace=True, drop=True)

In [3]:
# Load lists
with open('problem_dummies_subject_list.pickle', 'rb') as handle:
    subject_list = pickle.load(handle)
with open('joint_switch_spell_list.pickle', 'rb') as handle:
    spell_list = pickle.load(handle)

In [4]:
len(subject_list)
len(spell_list)

65160

12251

In [ ]:
# Import 
query = '''
SELECT * from ICHT_PROD.ICHT_COVID.EPISODES_DIAGNOSIS_2023
'''
cur = ctx.cursor().execute(query)
diagnosis_2023 = pd.DataFrame.from_records(iter(cur), columns=[x[0] for x in cur.description])
# Filter
diagnosis_2023 = diagnosis_2023[diagnosis_2023['SUBJECT'].isin(subject_list)]
diagnosis_2023 = diagnosis_2023[diagnosis_2023['SPELL_IDENTIFIER'].isin(spell_list)]
# Filter by uti
uti_diagnoses = diagnosis_2023[(diagnosis_2023['DIAGNOSIS_DESCRIPTION_ICD'].str.contains("urinary tract infection", case=False)) | \
                                  (diagnosis_2023['DIAGNOSIS_DESCRIPTION_ICD'].str.contains("pyelonephritis", case=False)) | \
                                  (diagnosis_2023['DIAGNOSIS_DESCRIPTION_SNOMED'].str.contains("urinary tract infection", case=False)) | \
                                  (diagnosis_2023['DIAGNOSIS_DESCRIPTION_SNOMED'].str.contains("pyelonephritis", case=False))]
# Filter by pneumonia
pneumonia_diagnoses = diagnosis_2023[(diagnosis_2023['DIAGNOSIS_DESCRIPTION_ICD'].str.contains("pneumonia", case=False)) | \
                                  (diagnosis_2023['DIAGNOSIS_DESCRIPTION_SNOMED'].str.contains("pneumonia", case=False))]

# Filter by sepsis
sepsis_diagnoses = diagnosis_2023[(diagnosis_2023['DIAGNOSIS_DESCRIPTION_ICD'].str.contains("sepsis", case=False)) | \
                                  (diagnosis_2023['DIAGNOSIS_DESCRIPTION_SNOMED'].str.contains("sepsis", case=False))]

# Filter by cellulitis
cellulitis_diagnoses = diagnosis_2023[(diagnosis_2023['DIAGNOSIS_DESCRIPTION_ICD'].str.contains("cellulitis", case=False)) | \
                                  (diagnosis_2023['DIAGNOSIS_DESCRIPTION_SNOMED'].str.contains("cellulitis", case=False))]

# Filter by bacteraemia
bacteraemia_diagnoses = diagnosis_2023[(diagnosis_2023['DIAGNOSIS_DESCRIPTION_ICD'].str.contains("blood stream infection", case=False)) | \
                                  (diagnosis_2023['DIAGNOSIS_DESCRIPTION_ICD'].str.contains("bacteraemia", case=False)) | \
                                  (diagnosis_2023['DIAGNOSIS_DESCRIPTION_SNOMED'].str.contains("blood stream infection", case=False)) | \
                                  (diagnosis_2023['DIAGNOSIS_DESCRIPTION_SNOMED'].str.contains("bacteraemia", case=False))]

uti_diagnoses.SUBJECT.nunique()
uti_diagnoses.SPELL_IDENTIFIER.nunique()

pneumonia_diagnoses.SUBJECT.nunique()
pneumonia_diagnoses.SPELL_IDENTIFIER.nunique()

sepsis_diagnoses.SUBJECT.nunique()
sepsis_diagnoses.SPELL_IDENTIFIER.nunique()

cellulitis_diagnoses.SUBJECT.nunique()
cellulitis_diagnoses.SPELL_IDENTIFIER.nunique()

bacteraemia_diagnoses.SUBJECT.nunique()
bacteraemia_diagnoses.SPELL_IDENTIFIER.nunique()

# Create binary columns
uti_diagnoses['uti'] = 1
pneumonia_diagnoses['pneumonia'] = 1
sepsis_diagnoses['sepsis'] = 1
cellulitis_diagnoses['cellulitis'] = 1

# Merge DataFrames
merged_df = uti_diagnoses[['SPELL_IDENTIFIER', 'uti']]
merged_df = merged_df.merge(pneumonia_diagnoses[['SPELL_IDENTIFIER', 'pneumonia']], how='outer')
merged_df = merged_df.merge(sepsis_diagnoses[['SPELL_IDENTIFIER', 'sepsis']], how='outer')
merged_df = merged_df.merge(cellulitis_diagnoses[['SPELL_IDENTIFIER', 'cellulitis']], how='outer')

# Fill NaN values with 0
merged_df = merged_df.fillna(0)
disease_df_2023 = merged_df.drop_duplicates()
disease_df_2023.reset_index(inplace=True, drop=True)
disease_df_2023

In [14]:
# Import 
query = '''
SELECT * from ICHT_PROD.ICHT_COVID.DEMOGRAPHIC
'''
cur = ctx.cursor().execute(query)
demographics = pd.DataFrame.from_records(iter(cur), columns=[x[0] for x in cur.description])

In [16]:
# Create episode lists 
episode_subject_list = episodes['SUBJECT'].unique().tolist()

In [17]:
# Filter
demographics = demographics[demographics['SUBJECT'].isin(episode_subject_list)]
# Filter for relevant columns 
demographics = demographics[['SUBJECT', 'GENDER_DESC', 'AGE', 'ETHNICITY_DESC', 'IMDDECIL']]
# Conver to int
demographics['AGE'] = pd.to_numeric(demographics['AGE'])
demographics['IMDDECIL'] = pd.to_numeric(demographics['IMDDECIL'])
# Reset index 
demographics.reset_index(inplace=True, drop=True)

In [19]:
# Filter for those over 18
demographics = demographics[demographics['AGE'] >= 18]

In [20]:
demographics.AGE.max()
demographics.AGE.min()
demographics.IMDDECIL.max()
demographics.IMDDECIL.min()

103

18

10.0

1.0

In [21]:
from sklearn import preprocessing

# Filling missing data, normalize features
min_max_scaler = preprocessing.MinMaxScaler()
data_scaled = min_max_scaler.fit_transform(demographics[['AGE', 'IMDDECIL']])
data_scaled = np.nan_to_num(data_scaled, nan=-1)

In [24]:
def grouped_ethincity_fun(a):
    if "mixed " in a.lower():
        return "Other"
    elif "white" in a.lower():
        return "White"
    elif "black" in a.lower():
        return "Black"
    elif "asian" in a.lower():
        return "Asian"
    elif "other " in a.lower():
        return "Other"
    else:
        return "Other"

In [ ]:
# Group race
demographics["grouped_ethincity"] = demographics.ETHNICITY_DESC.apply(lambda x: grouped_ethincity_fun(x))

In [26]:
# Create one hot
one_hot_demographics = pd.get_dummies(demographics[['GENDER_DESC', 'grouped_ethincity']])

In [28]:
# Create final df
final_demographics = pd.DataFrame()
final_demographics['SUBJECT'] = demographics['SUBJECT'] 
final_demographics[['AGE', 'IMDDECIL']] = data_scaled
final_demographics = pd.concat([final_demographics, one_hot_demographics], axis=1)

In [ ]:
# Import 
query = '''
SELECT * from ICHT_PROD.ICHT_COVID.DEMOGRAPHIC_2023
'''
cur = ctx.cursor().execute(query)
demographics_2023 = pd.DataFrame.from_records(iter(cur), columns=[x[0] for x in cur.description])
# Create episode lists 
episode_2023_subject_list = episodes_2023['SUBJECT'].unique().tolist()
# Filter
demographics_2023 = demographics_2023[demographics_2023['SUBJECT'].isin(episode_2023_subject_list)]
# Filter for relevant columns 
demographics_2023 = demographics_2023[['SUBJECT', 'GENDER_DESC', 'AGE', 'ETHNICITY_DESC', 'IMDDECIL']]
# Conver to int
demographics_2023['AGE'] = pd.to_numeric(demographics_2023['AGE'])
demographics_2023['IMDDECIL'] = pd.to_numeric(demographics_2023['IMDDECIL'])
# Reset index 
demographics_2023.reset_index(inplace=True, drop=True)
from sklearn import preprocessing
# Filling missing data, normalize features
min_max_scaler = preprocessing.MinMaxScaler()
data_scaled = min_max_scaler.fit_transform(demographics_2023[['AGE', 'IMDDECIL']])
data_scaled = np.nan_to_num(data_scaled, nan=-1)
# Group race
demographics_2023["grouped_ethincity"] = demographics_2023.ETHNICITY_DESC.apply(lambda x: grouped_ethincity_fun(x))
# Create one hot
one_hot_demographics = pd.get_dummies(demographics_2023[['GENDER_DESC', 'grouped_ethincity']])
# Create final df
final_demographics_2023 = pd.DataFrame()
final_demographics_2023['SUBJECT'] = demographics_2023['SUBJECT'] 
final_demographics_2023[['AGE', 'IMDDECIL']] = data_scaled
final_demographics_2023 = pd.concat([final_demographics_2023, one_hot_demographics], axis=1)
final_demographics_2023

In [12]:
demographics_2023.AGE.max()
demographics_2023.AGE.min()
demographics_2023.IMDDECIL.max()
demographics_2023.IMDDECIL.min()

104

18

10.0

1.0

In [12]:
# Save
#with open('problem_dummies_subject_list.pickle', 'wb') as handle:
#    pickle.dump(subject_list, handle)
#with open('joint_switch_spell_list.pickle', 'wb') as handle:
#    pickle.dump(spell_list, handle)

In [30]:
# Save
#episodes.to_csv('switch_data/chronic_switch_episodes.csv', index=False)
# Save
#disease_df.to_csv('switch_data/chronic_switch_disease.csv', index=False)
# Save
#final_demographics.to_csv('switch_data/chronic_switch_demographics.csv', index=False)

In [11]:
# Save
#episodes_2023.to_csv('switch_data/chronic_switch_episodes_2023.csv', index=False)

In [6]:
# Save
#disease_df_2023.to_csv('switch_data/chronic_switch_disease_2023.csv', index=False)

In [13]:
# Save
#final_demographics_2023.to_csv('switch_data/chronic_switch_demographics_2023.csv', index=False)

In [4]:
# Import
path = r'switch_data/chronic_switch_episodes.csv'
episodes = pd.read_csv(path)
# Import
path = r'switch_data/chronic_switch_episodes_2023.csv'
episodes_2023 = pd.read_csv(path)

In [10]:
# Create episode lists 
episode_subject_list = episodes['SUBJECT'].unique().tolist()
episode_spell_list = episodes['SPELL_IDENTIFIER'].unique().tolist()
# Create episode lists 
episode_2023_subject_list = episodes_2023['SUBJECT'].unique().tolist()
episode_2023_spell_list = episodes_2023['SPELL_IDENTIFIER'].unique().tolist()

In [8]:
# Import
path = r'switch_data/icare_switch_data_preprocessed.csv'
icare_df_preprocessed = pd.read_csv(path)
# Import
path = r'switch_data/icare_switch_data_preprocessed_2023.csv'
icare_df_preprocessed_2023 = pd.read_csv(path)

In [12]:
len(icare_df_preprocessed)
icare_df_preprocessed['SPELL_IDENTIFIER'].nunique()
len(icare_df_preprocessed_2023)
icare_df_preprocessed_2023['SPELL_IDENTIFIER'].nunique()
len(problem_dummies)
problem_dummies['SUBJECT'].nunique()

55888

10007

10507

2244

95157

65160

In [13]:
# Filter
icare_df_preprocessed = icare_df_preprocessed[icare_df_preprocessed['SPELL_IDENTIFIER'].isin(episode_spell_list)]
icare_df_preprocessed_2023 = icare_df_preprocessed_2023[icare_df_preprocessed_2023['SPELL_IDENTIFIER'].isin(episode_2023_spell_list)]
filtered_problem_dummies = problem_dummies[problem_dummies['SUBJECT'].isin(episode_subject_list)]

In [15]:
# Save
icare_df_preprocessed.to_csv('switch_data/chronic_switch_icare_df_preprocessed.csv', index=False)
# Save
icare_df_preprocessed_2023.to_csv('switch_data/chronic_switch_icare_df_preprocessed_2023.csv', index=False)
# Save
filtered_problem_dummies.to_csv('switch_data/chronic_switch_problem_dummies.csv', index=False)